### Training and testing with `scale_firing_rates`, and `LoihiSpikingRectifiedLinear()`.

In [1]:
import numpy as np
import nengo
import nengo_dl
import nengo_loihi
import tensorflow as tf

np.random.seed(0)
tf.random.set_seed(0)

In [2]:
(train_images, train_labels), (test_images, test_labels) = (
  tf.keras.datasets.mnist.load_data()
)
# Make it channels_first.
train_images, test_images = np.expand_dims(train_images, -1), np.expand_dims(test_images, -1)
train_images, test_images = np.moveaxis(train_images, -1, 1), np.moveaxis(test_images, -1, 1)

# Flatten images and add time dimension.
train_images = train_images.reshape((train_images.shape[0], 1, -1))
train_labels = train_labels.reshape((train_labels.shape[0], 1, -1))
test_images = test_images.reshape((test_images.shape[0], 1, -1))
test_labels = test_labels.reshape((test_labels.shape[0], 1, -1))

# Create the TF Model

In [3]:
inp = tf.keras.Input(shape=(1, 28, 28), name="input")

to_spikes = tf.keras.layers.Conv2D(
  filters=3, # 3 RGB Neurons per pixel.
  kernel_size=(1, 1), strides=(1, 1), activation=tf.nn.relu, use_bias=False, # Default is True.
  data_format="channels_first", name="to-spikes")(inp)

conv0 = tf.keras.layers.Conv2D(
  filters=8, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, use_bias=False,
  data_format="channels_first", name="conv0")(to_spikes)

# maxp0 = tf.keras.layers.MaxPool2D(
#   pool_size=(2, 2), padding="valid", data_format="channels_first", name="MaxPool0")(conv0)

conv1 = tf.keras.layers.Conv2D(
  filters=16, kernel_size=(3, 3), strides=(2, 2), activation=tf.nn.relu, use_bias=False,
  data_format="channels_first", name="conv1")(conv0)

flatten = tf.keras.layers.Flatten(name="flatten")(conv1)
dense0 = tf.keras.layers.Dense(64, activation=tf.nn.relu, name="dense0")(flatten)

#dense1 = tf.keras.layers.Dense(10, activation="softmax", name="dense1")(dense0) # Results in formation of TensorNode
                                                                                # which isn't supported on Loihi.
dense1 = tf.keras.layers.Dense(10, name="dense1")(dense0)

model = tf.keras.Model(inputs=inp, outputs=dense1)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1, 28, 28)]       0         
_________________________________________________________________
to-spikes (Conv2D)           (None, 3, 28, 28)         3         
_________________________________________________________________
conv0 (Conv2D)               (None, 8, 26, 26)         216       
_________________________________________________________________
conv1 (Conv2D)               (None, 16, 12, 12)        1152      
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense0 (Dense)               (None, 64)                147520    
_________________________________________________________________
dense1 (Dense)               (None, 10)                650   

# Convert TF Model to Nengo DL Model

In [4]:
def _get_nengo_dl_model(is_test=False, **kwargs):
  converter = nengo_dl.Converter(model, **kwargs)
#   if is_test:
#     #converter.net._connections[2].synapse=nengo.Lowpass(0.005)
#     for conn in converter.net._connections:
#       print(conn, conn.synapse)
  return converter

In [5]:
def train(params_file="./attempting_TN_MP_loihineurons_8_16", epochs=1, **kwargs):
  converter = _get_nengo_dl_model(**kwargs)
  
  with nengo_dl.Simulator(converter.net, seed=0, minibatch_size=200) as sim:
    sim.compile(
      optimizer=tf.optimizers.Adam(0.001),
      loss={
        converter.outputs[dense1]: tf.losses.SparseCategoricalCrossentropy(from_logits=True)
      },
      metrics={converter.outputs[dense1]: tf.metrics.sparse_categorical_accuracy},
    )
    sim.fit(
      {converter.inputs[inp]: train_images},
      {converter.outputs[dense1]: train_labels},
      epochs=epochs,
    )
    
    # save the parameters to file.
    sim.save_params(params_file)

In [6]:
def test_network(activation, params_file="./attempting_TN_MP_loihineurons_8_16", n_steps=30,
                 scale_firing_rates=1, synapse=None, n_test=100):
  nengo_converter = _get_nengo_dl_model(
    is_test=True,
    scale_firing_rates=scale_firing_rates, 
    swap_activations={tf.nn.relu: activation},
    synapse=synapse
  )
  
  tiled_test_images = np.tile(test_images[:n_test], (1, n_steps, 1))
  # Speed up simulation.
  with nengo_converter.net:
    nengo_dl.configure_settings(stateful=False)
    
  # Build network, load in trained weights, do inference.
  with nengo_dl.Simulator(
      nengo_converter.net, minibatch_size=20, progress_bar=False) as sim:
    sim.load_params(params_file)
    data = sim.predict({nengo_converter.inputs[inp]: tiled_test_images})
    
  test_predictions = np.argmax(data[nengo_converter.outputs[dense1]][:, -1], axis=-1)
  print("Nengo DL Test Acc: %s" % (100 * np.mean(test_predictions == test_labels[:n_test, 0, 0])))

# Train Nengo-DL Model with `LoihiSpikingRectifiedLinear()` neurons

In [7]:
train(
  params_file="./attempting_TN_MP_loihineurons_8_16",
  epochs=4,
  swap_activations={tf.nn.relu: nengo_loihi.neurons.LoihiSpikingRectifiedLinear()},
  scale_firing_rates=100,
)

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/4
300/300 [==============================] - 2s 8ms/step - loss: 0.2512 - probe_loss: 0.2512 - probe_sparse_categorical_accuracy: 0.9247
Epoch 2/4
300/300 [==============================] - 2s 7ms/step - loss: 0.0851 - probe_loss: 0.0851 - probe_sparse_categorical_accuracy: 0.9740
Epoch 3/4
300/300 [==============================] - 2s 7ms/step - loss: 0.0607 - probe_loss: 0.0607 - probe_sparse_categorical_accuracy: 0.9811
Epoch 4/4
300/300 [==============================] - 2s 7ms/step - loss: 0.0487 - probe_loss: 0.0487 - probe_sparse_categorical_accuracy: 0.9844


# Test Nengo-DL Model with `LoihiSpikingRectifiedLinear()` neurons

In [8]:
test_network(
  activation=nengo_loihi.neurons.LoihiSpikingRectifiedLinear(),
  scale_firing_rates=100,
  synapse=0.005
)

Nengo DL Test Acc: 100.0


# Convert TF Model to NengoLoihi Model

In [9]:
pres_time = 0.03
n_test = 5

nengo_converter_2 = _get_nengo_dl_model(
    is_test=True,
    scale_firing_rates=100, 
    swap_activations={tf.nn.relu: nengo_loihi.neurons.LoihiSpikingRectifiedLinear()},
    synapse=0.005
  )
nengo_input_2 = nengo_converter_2.inputs[inp]
nengo_output_2 = nengo_converter_2.outputs[dense1]

net_2 = nengo_converter_2.net
with nengo_dl.Simulator(net_2) as sim_2:
  sim_2.load_params("./attempting_TN_MP_loihineurons_8_16")
  sim_2.freeze_params(net_2)

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


In [13]:
converter_layers = nengo_converter_2.model.layers
print(converter_layers)

[<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x2b44c8544610>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x2b44c856ea50>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x2b44cb9f7b10>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x2b44c847d190>, <tensorflow.python.keras.layers.core.Flatten object at 0x2b44cbc87990>, <tensorflow.python.keras.layers.core.Dense object at 0x2b44cbc893d0>, <tensorflow.python.keras.layers.core.Dense object at 0x2b44c8476510>]


In [14]:
conv0_shape = converter_layers[2].output.shape[1:]
conv1_shape = converter_layers[3].output.shape[1:] # 4 with MP layers.
dense0_shape = converter_layers[5].output.shape[1:] # 6 with MP layers.

tuple(conv0_shape), tuple(conv1_shape), tuple(dense0_shape)

((8, 26, 26), (16, 12, 12), (64,))

# Configure the NengoLoihi settings/block shapes, etc.

In [15]:
converter_layers = nengo_converter_2.model.layers
with net_2:
  nengo_input_2.output = nengo.processes.PresentInput(
    test_images, presentation_time=pres_time
  )
  nengo_loihi.add_params(net_2)
  
  # Set the to_spikes layer Off-Chip.
  net_2.config[nengo_converter_2.layers[to_spikes].ensemble].on_chip = False
  # Set the maxp0 layer Off-Chip.
  # net_2.config[nengo_converter_2.layers[maxp0].ensemble].on_chip = False
  
  conv0_shape = tuple(converter_layers[2].output.shape[1:])
  conv1_shape = tuple(converter_layers[3].output.shape[1:])
  dense0_shape = tuple(converter_layers[5].output.shape[1:])
  
  net_2.config[
    nengo_converter_2.layers[conv0].ensemble
  ].block_shape = nengo_loihi.BlockShape((4, 16, 16), conv0_shape)
  net_2.config[nengo_converter_2.layers[conv1].ensemble].block_shape = (
      nengo_loihi.BlockShape((8, 11, 11), conv1_shape))
  net_2.config[nengo_converter_2.layers[dense0].ensemble].block_shape = (
      nengo_loihi.BlockShape((32,), dense0_shape))

# Build the NengoLoihi Network and Run it!

In [16]:
with nengo_loihi.Simulator(net_2, target="sim") as loihi_sim:
  loihi_sim.run(n_test * pres_time)
  
  # get output (last timestep of each presentation period)
  pres_steps = int(round(pres_time / loihi_sim.dt))
  output = loihi_sim.data[nengo_output_2][pres_steps - 1 :: pres_steps]
  
  # compute Loihi accuracy
  loihi_predictions = np.argmax(output, axis=-1)
  correct = 100 * np.mean(loihi_predictions == test_labels[:n_test, 0, 0])
  print("Nengo Loihi Accuracy: %s" % correct)

Nengo Loihi Accuracy: 100.0
